<a href="https://colab.research.google.com/github/panditamey1/NLP_chatbot_gradio_gpt3/blob/main/doc_retrieval_haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

nvidia-smi

Tue Nov 29 07:11:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-01b1jau9/farm-haystack_8712dc2ea49946a98261b88bc9d4367d
  Resolved https://github.com/deepset-ai/haystack.git to commit 839eef66958578efe7135f54e14f3fe462c78f43
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-01b1jau9/farm-haystack_8712dc2ea49946a98261b88bc9d4367d


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [4]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()


In [5]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [6]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [7]:
import time
time.sleep(30)

In [8]:
import os
from haystack.document_stores import ElasticsearchDocumentStore


In [9]:
# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [10]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


In [11]:
doc_dir = "/content/"
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)


INFO:haystack.utils.preprocessing:Converting /content/Fire engine accessway and fire engine access road.txt
INFO:haystack.utils.preprocessing:Converting /content/Electromagnetic or electromechanical locking device.txt
INFO:haystack.utils.preprocessing:Converting /content/Fire.txt
INFO:haystack.utils.preprocessing:Converting /content/External exit staircase.txt
INFO:haystack.utils.preprocessing:Converting /content/Habitable floor.txt
INFO:haystack.utils.preprocessing:Converting /content/Ventilation openings.txt
INFO:haystack.utils.preprocessing:Converting /content/Obstruction.txt
INFO:haystack.utils.preprocessing:Converting /content/basement.txt
INFO:haystack.utils.preprocessing:Converting /content/elasticsearch-7.9.2/LICENSE.txt
INFO:haystack.utils.preprocessing:Converting /content/elasticsearch-7.9.2/NOTICE.txt
INFO:haystack.utils.preprocessing:Converting /content/elasticsearch-7.9.2/jdk/conf/security/policy/README.txt
INFO:haystack.utils.preprocessing:Converting /content/elasticsearc

In [12]:
print(docs[:3])

[<Document: {'content': 'Fire engine accessway and fire engine access road\n(3) PG II buildings exceeding 10m habitable height\nFor a building under PG II that exceeds the habitable height of 10m, all of the\nfollowing shall be complied with:\na. A fire engine accessway/fire engine access road shall be provided within a travel\ndistance of 18m to the entrance of all exit staircases where the landing valves (dry\nor wet riser) are provided in accordance with Cl.6.2.2b.\nb. A fire engine accessway of at least ¼ length of perimeter (minimum 15m),\nwhichever is greater, shall be provided to access at least one entire façade of each\nblock and shall be located at a distance of at least 2m and at most 10m away from\nthe façade of the building. This is to facilitate rescue with direct access to unit\nwindows (excluding exit staircase, smoke-free approach to exit staircase). For\nthe purpose of computing the extent of perimeter for fire engine accessway,\nonly the external facade perimeters of

In [13]:
document_store.write_documents(docs)

In [14]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [15]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [16]:
from haystack.nodes import TransformersReader


In [17]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [18]:
prediction = pipe.run(
    query="Fire safety?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:02<00:00,  2.72s/ Batches]


In [19]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'obstructions', 'type': 'extractive', 'score': 0.16412723064422607, 'context': 're engine accessways/fire engine access roads shall be kept clear of\nobstructions at all times. Plants, trees or other fixtures shall not obstruct the', 'offsets_in_document': [{'start': 71, 'end': 83}], 'offsets_in_context': [{'start': 69, 'end': 81}], 'document_id': '5b88b3100f1bab1b18acf19198411a1b', 'meta': {'name': 'Obstruction.txt'}}>,
             <Answer {'answer': 'A fire engine accessway/fire engine access road shall be provided within a travel\ndistance of 18m to the entrance of all exit staircases', 'type': 'extractive', 'score': 0.11052802950143814, 'context': 'th:\na. A fire engine accessway/fire engine access road shall be provided within a travel\ndistance of 18m to the entrance of all exit staircases where ', 'offsets_in_document': [{'start': 218, 'end': 354}], 'offsets_in_context': [{'start': 7, 'end': 143}], 'document_id': 'e086515f4107e52631fa0894411de9ff

In [20]:
from haystack.utils import print_answers

# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(prediction, details="minimum")


Query: Fire safety?
Answers:
[   {   'answer': 'obstructions',
        'context': 're engine accessways/fire engine access roads shall be '
                   'kept clear of\n'
                   'obstructions at all times. Plants, trees or other fixtures '
                   'shall not obstruct the'},
    {   'answer': 'A fire engine accessway/fire engine access road shall be '
                  'provided within a travel\n'
                  'distance of 18m to the entrance of all exit staircases',
        'context': 'th:\n'
                   'a. A fire engine accessway/fire engine access road shall '
                   'be provided within a travel\n'
                   'distance of 18m to the entrance of all exit staircases '
                   'where '},
    {   'answer': 'storage of flammable',
        'context': 'eline service corridors in Singapore\n'
                   'Fire safety requirement for storage of flammable\n'
                   'The safe storage of flammable liquid